# 信用情報の分析

## 信用情報

Kaggleの Home Credit Default Risk コンペティションを活用し、実データに近いものに対し、自ら課題を設定して分析する練習を行います。<br>

Home Credit Default Risk | Kaggle<br>

Week4では機械学習手法を用いて学習・推定を行います。その準備としてWeek3でデータ探索（EDA）を行います。<br>

【問題1】コンペティション内容の把握<br>
コンペティションのOverviewページ読み、「Home Credit Default Risk」はどのようなコンペティションか、以下の観点からレポートしてください。<br>

Home Creditはどのような企業？<br>
このコンペティションでは何を予測する？<br>
それを予測することで企業が得られるメリットは何？<br>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy import stats
%matplotlib inline


train = pd.read_csv('./Home_credit_Default_Risk/application_train.csv')
pd.set_option('display.max_columns', train.shape[1])
pd.set_option('display.max_rows', 100)
train.head()

1.Home Creditはどのような企業？<br>
Home Credit社は世界的に銀行口座を持たないような顧客に対する小口ローン融資を9ヵ国で展開している消費者金融。<br>
顧客の購買履歴に基づく融資を行っている。<br>
2.このコンペティションでは何を予測する？<br>
顧客の返済可・不可を予測する<br>
3.それを予測することで企業が得られるメリットは何？<br>
適切な融資額を決定することができる為、貸し倒れリスクを減らすことができる。<br>

今回のような2値分類を行うタスクの場合のEDAの心構え<br>
与えられてた訓練データから、データが所属するクラス毎に、どんな特徴が存在するのかを特徴量から見出し、新規の特徴量を作成したり、<br>
似た特徴量を減らしたり、くっつけて新しい特徴量を生み出したりすること<br>

【問題2】データの概観の把握<br>
データの概観を把握するために以下のようなことを行ってください。<br>

・.head()、.info()、describe()などを使う<br>
・欠損値の有無を見る<br>
・クラスの割合がわかるグラフを描く<br>

それぞれ結果に対する説明や考察も行ってください。<br>


In [ ]:
# .head()、.info()、describe()などを使う
train.head()

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
# 欠損値の有無を見る
n_missing = train.isnull().sum() 
print("欠損数が{}より多い特徴量の一覧\n{}".format(0, n_missing[n_missing>0]))

In [ ]:
# データタイプ毎に特徴量を分割
numerical_feats = train.dtypes[train.dtypes != 'object'].index
categorical_feats = train.dtypes[train.dtypes == 'object'].index

print(numerical_feats, len(numerical_feats))
print(categorical_feats, len(categorical_feats))

In [ ]:
# 数値データだけ表示
train[numerical_feats]

In [ ]:
# カテゴリーデータだけ表示
train[categorical_feats]

In [ ]:
# クラスの割合がわかるグラフを描く
train.TARGET.value_counts().plot(kind='pie', autopct='%1.1f%%')

In [ ]:
# データ全体の返済完了者、未完了者の割合を確認
col = 'TARGET'
temp = train[col].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })

plt.figure(figsize = (6,6))
plt.title('Application loans repayed - Pay_ok[{}]'.format(col))
sns.set_color_codes("pastel")
sns.barplot(x = 'labels', y="values", data=df)
plt.show()

In [ ]:
# 年収ベースで見たときの外れ値の確認
thresh = 10000000
train[train['AMT_INCOME_TOTAL'] > thresh]
# income_totalの外れ値　こいつは外していいのでは？
# 年収1億ドルで労働者で返済できませんでした。は明らかにおかしいし、そもそも何故借金の必要が？

考察：<br>
0が、支払いに問題ない人、1が、支払いに問題がある（デフォルト）人で、その割合が0に偏ってしまっている。<br>
そこで、1に分類されている人たちの中の共通点を探してあげる必要があると考える。<br>

【問題3】課題設定<br>
データの概観を見たことを元に、自分なりの課題・疑問を複数設定してください。<br>

1.target:0とtarget:1での年収の分布を見る<br>
2.貸し倒れてしまう人の要因<br>
3.貸倒ない人の行動、要因は？<br>

【問題4】データ探索

返済できている人、そうでない人との間に年収において差があるのではないかと考えたので、<br>
データを分けて分布を見てみる

In [ ]:
pay_ok = train[train['TARGET'] == 0]
default = train[train['TARGET'] == 1]

sns.distplot(pay_ok['AMT_INCOME_TOTAL'], bins=500, fit=norm)
fig = plt.figure()
res = stats.probplot(pay_ok['AMT_INCOME_TOTAL'], plot=plt)

値が極端に寄ってしまっていて見づらいので対数変換

In [ ]:
# 返済完了者データ：対数変換後
sns.distplot(pay_ok['AMT_INCOME_TOTAL'].apply(np.log), bins=100, fit=norm)
fig = plt.figure()
res = stats.probplot(pay_ok['AMT_INCOME_TOTAL'].apply(np.log), plot=plt)

In [ ]:
# 返済完了者データ：対数変換前
sns.distplot(default['AMT_INCOME_TOTAL'], bins=500, fit=norm)
fig = plt.figure()
res = stats.probplot(default['AMT_INCOME_TOTAL'], plot=plt)

こちらも同様

In [ ]:
# 返済未完了者データ：対数変換後
sns.distplot(default['AMT_INCOME_TOTAL'].apply(np.log), bins=500, fit=norm)
fig = plt.figure()
res = stats.probplot(default['AMT_INCOME_TOTAL'].apply(np.log), plot=plt)

QQプロット的には、年収はで正規性が確保できていると言えるので、予測には使えそう。<br>
同じヒストグラムで分布を比較してみる

In [ ]:
a = pay_ok['AMT_INCOME_TOTAL'].apply(np.log)
b = default['AMT_INCOME_TOTAL'].apply(np.log)
plt.figure(figsize=(10,10))
plt.hist([a, b], bins=100, label=['ok', 'default'])
plt.legend(loc='upper left')
plt.show()

返済完了できている人とそうでない人との分布にそこまでの差はなさそう。<br>
もしかしたら年収単体では意味をなさないかも？<br>

データにyes,noや0, 1のデータが多いので、barplotで返済できている人、そうでない人に特徴がないか確認してみる<br>
まずは融資方法

In [ ]:
col = "NAME_CONTRACT_TYPE"
temp = pay_ok[col].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })

temp2 = default[col].value_counts()
df2 = pd.DataFrame({'labels': temp2.index,
                   'values': temp2.values
                  })
plt.figure(figsize = (6,6))
plt.title('Application loans repayed - Pay_ok[{}]'.format(col))
sns.set_color_codes("pastel")
sns.barplot(x = 'labels', y="values", data=df)

plt.figure(figsize = (6,6))
plt.title('Application loans repayed - default[{}]'.format(col))
sns.set_color_codes("pastel")
sns.barplot(x = 'labels', y="values", data=df2)

plt.show()

df['CONTRACT_TYPE_ratio'] = df['values'] / df['values'].sum()
df2['CONTRACT_TYPE_ratio'] = df2['values'] / df2['values'].sum()

print(df)
print(df2)

比率を見てみると、返済できている人にも、そうでない人にもリボルビングの人はいるし、キャッシュローンの人も一定数存在している<br>
あまり有意ではなさそう。<br>
<br>
その他のカテゴリーデータでどんな感じか確認

In [ ]:
for col in categorical_feats:
    temp = pay_ok[col].value_counts()
    df = pd.DataFrame({'labels': temp.index,
                       'values': temp.values
                      })
    temp = default[col].value_counts()
    df2 = pd.DataFrame({'labels': temp.index,
                       'values': temp.values
                      })
    plt.subplot(1, 2, 1)
#     plt.figure(figsize=(5,5))
    plt.title('Pay_ok[{}]'.format(col))
    sns.set_color_codes("pastel")
    sns.barplot(x = 'labels', y="values", data=df)
    plt.xticks(rotation=90)
    
    plt.subplot(1, 2, 2)
#     plt.figure(figsize=(5,5))
    plt.title('default[{}]'.format(col))
    sns.set_color_codes("pastel")
    sns.barplot(x = 'labels', y="values", data=df2)
    plt.xticks(rotation=90)
    
    df[col + '_ratio'] = df['values'] / df['values'].sum()
    df2[col + '_ratio'] = df2['values'] / df2['values'].sum()
    
    print('#'*50)

    plt.tight_layout()
    plt.show()
    
    print(df)
    print('-'*50)
    print(df2)

カテゴリーデータにおける欠損値は現時点で未処理ではあるものの、返済できている人、そうでない人との間で特徴の偏りが見られなかった。<br>

数値データで、特徴量ごとの分布がどうなっているかを見てみる

In [ ]:
for col in numerical_feats:
    plt.subplot(1, 2, 1)
    plt.title('Pay_ok[{}]'.format(col))
#     sns.set_color_codes("pastel")
#     sns.distplot(pay_ok[col], bins=500, fit=norm)
    pay_ok[col].hist(bins=100)
    plt.xticks(rotation=90)
    
    plt.subplot(1, 2, 2)
#     plt.figure(figsize=(5,5))
    plt.title('default[{}]'.format(col))
#     sns.set_color_codes("pastel")
#     sns.distplot(default[col], bins=500, fit=norm)
    default[col].hist(bins=100)
    plt.xticks(rotation=90)

    plt.tight_layout()
    plt.show()
    
    print('#'*50)

In [ ]:
a = pd.DataFrame([])
a['credit_ratio'] = train['AMT_CREDIT'] / train['AMT_INCOME_TOTAL']
plt.xlim(0, 25)
plt.hist(a['credit_ratio'], bins=100)

借入日から何日前に生まれたかと言うDAY_BIRTHの項目で、返済できている人とそうでない人の分布に明確な違いがあった。比較的若者に、返済能力が低い傾向が見られる。<br>

In [ ]:
pay_ok[pay_ok['DAYS_BIRTH'] < 15000]
a = pay_ok.copy()
a['DAYS_BIRTH'] = abs(a['DAYS_BIRTH'] / 365)
a['DAYS_BIRTH'].hist(bins=20)

In [ ]:
how_old_are_you = default.copy()
how_old_are_you['DAYS_BIRTH'] = abs(how_old_are_you['DAYS_BIRTH'] / 365)
how_old_are_you['DAYS_BIRTH'].hist(bins=20)

今回のタスクが0,1の予測なので、目的変数と説明変数間の相関に意味はないと思うが、説明変数間での相関がどうなっているのかを確認する。<br>
事前課題で作成した関数をもとに、返済できている人、そうでない人ごとに特徴量の相関がないかチェック。<br>

In [ ]:
ok_corr = train[train.TARGET == 0].corr()
default_corr = train[train['TARGET'] == 1].corr()

# 選んだ10個の特徴量の中でお互いの相関係数が高い組み合わせを3つ探し出す。
def calc_corr(df):
    # 相関係数行列を作成
    corr_mat = df.corr(method='pearson')

    # 行（列）サイズを取得
    n = corr_mat.shape[0]

    # 項目名を取得
    columns = corr_mat.columns.tolist()

    # 変数名1, 変数名2, 値を一つの配列に入れたものを作成
    # 相関係数行列の下三角部分（対角成分除く）だけ
    corr_ary = []
    var1_ary = []
    var2_ary = []
    for i in range(n):
        for j in range(i):
#             print(i, j)
            if i == j:
                continue
            corr_ary.append(corr_mat.iloc[i,j])
            var1_ary.append(columns[i])
            var2_ary.append(columns[j])

    # dfにする
    df_new = pd.DataFrame([])
    df_new["var1"] = var1_ary
    df_new["var2"] = var2_ary
    df_new["corr"] = corr_ary

    return df_new

ok = calc_corr(ok_corr)
ok.sort_values('corr', ascending=False).head(100)

In [ ]:
no = calc_corr(default_corr)
no.sort_values('corr', ascending=False).head(100)

これだけだと相関は現時点で使い物にならなさそう。<br>
適切な相関値が求まっているとは思えない。<br>

# 以下個人的な調査学習
## Home Credit Default Risk 1位ソリューション
### null importanceによる特徴量選択

Features Selection with NUll Importance<br>
URL:https://www.kaggle.com/ogrellier/feature-selection-with-null-importances<br>
null importanceについての解説記事<br>
https://qiita.com/trapi/items/1d6ede5d492d1a9dc3c9<br>
2位のソリューション：https://github.com/KazukiOnodera/Home-Credit-Default-Risk<br>

In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import time
from lightgbm import LGBMClassifier
import lightgbm as lgb

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
%matplotlib inline

import warnings
warnings.simplefilter('ignore', UserWarning)

import gc
gc.enable()

In [ ]:
data = pd.read_csv('./Home_credit_Default_Risk/application_train.csv')

categoricaal_feats = [
    f for f in data.columns if data[f].dtype == 'object'
]

categorical_feats
for f_ in categorical_feats:
    data[f_], _ = pd.factorize(data[f_])
    # Set feature type as categorical
    data[f_] = data[f_].astype('category')

In [ ]:
def get_feature_importances(data, shuffle, seed=None):
    # Gather real features
    train_features = [f for f in data if f not in ['TARGET', 'SK_ID_CURR']]
    # Go over fold and keep track of CV score (train and valid) and feature importances
    
    # Shuffle target if required
    y = data['TARGET'].copy()
    if shuffle:
        # Here you could as well use a binomial distribution
        y = data['TARGET'].copy().sample(frac=1.0)
    
    # Fit LightGBM in RF mode, yes it's quicker than sklearn RandomForest
    dtrain = lgb.Dataset(data[train_features], y, free_raw_data=False, silent=True)
    lgb_params = {
        'objective': 'binary',
        'boosting_type': 'rf',
        'subsample': 0.623,
        'colsample_bytree': 0.7,
        'num_leaves': 127,
        'max_depth': 8,
        'seed': seed,
        'bagging_freq': 1,
        'n_jobs': 4
    }
    
    # Fit the model
    clf = lgb.train(params=lgb_params, train_set=dtrain, num_boost_round=200, categorical_feature=categorical_feats)

    # Get feature importances
    imp_df = pd.DataFrame()
    imp_df["feature"] = list(train_features)
    imp_df["importance_gain"] = clf.feature_importance(importance_type='gain')
    imp_df["importance_split"] = clf.feature_importance(importance_type='split')
    imp_df['trn_score'] = roc_auc_score(y, clf.predict(data[train_features]))
    
    return imp_df

In [ ]:
# Seed the unexpected randomness of this world
np.random.seed(123)
# Get the actual importance, i.e. without shuffling
actual_imp_df = get_feature_importances(data=data, shuffle=False)

In [ ]:
actual_imp_df

gain:評価基準をどれだけ改善させたることができる(できた)のかという値<br>
split:ツリーの分割にその特徴量が使われた回数をカウントしたもの<br>

In [ ]:
null_imp_df = pd.DataFrame()
nb_runs = 80
import time
start = time.time()
dsp = ''
for i in range(nb_runs):
    # Get current run importances
    imp_df = get_feature_importances(data=data, shuffle=True)
    imp_df['run'] = i + 1 
    # Concat the latest importances with the old ones
    null_imp_df = pd.concat([null_imp_df, imp_df], axis=0)
    # Erase previous message
    for l in range(len(dsp)):
        print('\b', end='', flush=True)
    # Display current run and time used
    spent = (time.time() - start) / 60
    dsp = 'Done with %4d of %4d (Spent %5.1f min)' % (i + 1, nb_runs, spent)
    print(dsp, end='', flush=True)

In [ ]:
null_imp_df

In [ ]:
def display_distributions(actual_imp_df_, null_imp_df_, feature_):
    plt.figure(figsize=(13, 6))
    gs = gridspec.GridSpec(1, 2)
    # Plot Split importances
    ax = plt.subplot(gs[0, 0])
    a = ax.hist(null_imp_df_.loc[null_imp_df_['feature'] == feature_, 'importance_split'].values, label='Null importances')
    ax.vlines(x=actual_imp_df_.loc[actual_imp_df_['feature'] == feature_, 'importance_split'].mean(), 
               ymin=0, ymax=np.max(a[0]), color='r',linewidth=10, label='Real Target')
    ax.legend()
    ax.set_title('Split Importance of %s' % feature_.upper(), fontweight='bold')
    plt.xlabel('Null Importance (split) Distribution for %s ' % feature_.upper())
    # Plot Gain importances
    ax = plt.subplot(gs[0, 1])
    a = ax.hist(null_imp_df_.loc[null_imp_df_['feature'] == feature_, 'importance_gain'].values, label='Null importances')
    ax.vlines(x=actual_imp_df_.loc[actual_imp_df_['feature'] == feature_, 'importance_gain'].mean(), 
               ymin=0, ymax=np.max(a[0]), color='r',linewidth=10, label='Real Target')
    ax.legend()
    ax.set_title('Gain Importance of %s' % feature_.upper(), fontweight='bold')
    plt.xlabel('Null Importance (gain) Distribution for %s ' % feature_.upper())
    
display_distributions(actual_imp_df_=actual_imp_df, null_imp_df_=null_imp_df, feature_='LIVINGAPARTMENTS_AVG')

In [ ]:
display_distributions(actual_imp_df_=actual_imp_df, null_imp_df_=null_imp_df, feature_='CODE_GENDER')

In [ ]:
display_distributions(actual_imp_df_=actual_imp_df, null_imp_df_=null_imp_df, feature_='EXT_SOURCE_1')

In [ ]:
display_distributions(actual_imp_df_=actual_imp_df, null_imp_df_=null_imp_df, feature_='EXT_SOURCE_2')

In [ ]:
display_distributions(actual_imp_df_=actual_imp_df, null_imp_df_=null_imp_df, feature_='EXT_SOURCE_3')

In [ ]:
feature_scores = []
for _f in actual_imp_df['feature'].unique():
    f_null_imps_gain = null_imp_df.loc[null_imp_df['feature'] == _f, 'importance_gain'].values
    f_act_imps_gain = actual_imp_df.loc[actual_imp_df['feature'] == _f, 'importance_gain'].mean()
    gain_score = np.log(1e-10 + f_act_imps_gain / (1 + np.percentile(f_null_imps_gain, 75)))  # Avoid didvide by zero
    f_null_imps_split = null_imp_df.loc[null_imp_df['feature'] == _f, 'importance_split'].values
    f_act_imps_split = actual_imp_df.loc[actual_imp_df['feature'] == _f, 'importance_split'].mean()
    split_score = np.log(1e-10 + f_act_imps_split / (1 + np.percentile(f_null_imps_split, 75)))  # Avoid didvide by zero
    feature_scores.append((_f, split_score, gain_score))

scores_df = pd.DataFrame(feature_scores, columns=['feature', 'split_score', 'gain_score'])

plt.figure(figsize=(16, 16))
gs = gridspec.GridSpec(1, 2)
# Plot Split importances
ax = plt.subplot(gs[0, 0])
sns.barplot(x='split_score', y='feature', data=scores_df.sort_values('split_score', ascending=False).iloc[0:70], ax=ax)
ax.set_title('Feature scores wrt split importances', fontweight='bold', fontsize=14)
# Plot Gain importances
ax = plt.subplot(gs[0, 1])
sns.barplot(x='gain_score', y='feature', data=scores_df.sort_values('gain_score', ascending=False).iloc[0:70], ax=ax)
ax.set_title('Feature scores wrt gain importances', fontweight='bold', fontsize=14)
plt.tight_layout()

In [ ]:
null_imp_df.to_csv('null_importances_distribution_rf.csv')
actual_imp_df.to_csv('actual_importances_ditribution_rf.csv')

関係のない特徴量を削除した場合の影響度の確認

In [ ]:
correlation_scores = []
for _f in actual_imp_df['feature'].unique():
    f_null_imps = null_imp_df.loc[null_imp_df['feature'] == _f, 'importance_gain'].values
    f_act_imps = actual_imp_df.loc[actual_imp_df['feature'] == _f, 'importance_gain'].values
    gain_score = 100 * (f_null_imps < np.percentile(f_act_imps, 25)).sum() / f_null_imps.size
    f_null_imps = null_imp_df.loc[null_imp_df['feature'] == _f, 'importance_split'].values
    f_act_imps = actual_imp_df.loc[actual_imp_df['feature'] == _f, 'importance_split'].values
    split_score = 100 * (f_null_imps < np.percentile(f_act_imps, 25)).sum() / f_null_imps.size
    correlation_scores.append((_f, split_score, gain_score))

corr_scores_df = pd.DataFrame(correlation_scores, columns=['feature', 'split_score', 'gain_score'])

fig = plt.figure(figsize=(16, 16))
gs = gridspec.GridSpec(1, 2)
# Plot Split importances
ax = plt.subplot(gs[0, 0])
sns.barplot(x='split_score', y='feature', data=corr_scores_df.sort_values('split_score', ascending=False).iloc[0:70], ax=ax)
ax.set_title('Feature scores wrt split importances', fontweight='bold', fontsize=14)
# Plot Gain importances
ax = plt.subplot(gs[0, 1])
sns.barplot(x='gain_score', y='feature', data=corr_scores_df.sort_values('gain_score', ascending=False).iloc[0:70], ax=ax)
ax.set_title('Feature scores wrt gain importances', fontweight='bold', fontsize=14)
plt.tight_layout()
plt.suptitle("Features' split and gain scores", fontweight='bold', fontsize=16)
fig.subplots_adjust(top=0.93)

In [ ]:
corr_scores_df

In [ ]:
def score_feature_selection(df=None, train_features=None, cat_feats=None, target=None):
    # Fit LightGBM 
    dtrain = lgb.Dataset(df[train_features], target, free_raw_data=False, silent=True)
    lgb_params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'learning_rate': .1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'num_leaves': 31,
        'max_depth': -1,
        'seed': 13,
        'n_jobs': 4,
        'min_split_gain': .00001,
        'reg_alpha': .00001,
        'reg_lambda': .00001,
        'metric': 'auc'
    }
    
    # Fit the model
    hist = lgb.cv(
        params=lgb_params, 
        train_set=dtrain, 
        num_boost_round=2000,
        categorical_feature=cat_feats,
        nfold=5,
        stratified=True,
        shuffle=True,
        early_stopping_rounds=50,
        verbose_eval=0,
        seed=17
    )
    # Return the last mean / std values 
    return hist['auc-mean'][-1], hist['auc-stdv'][-1]

# features = [f for f in data.columns if f not in ['SK_ID_CURR', 'TARGET']]
# score_feature_selection(df=data[features], train_features=features, target=data['TARGET'])

for threshold in [0, 10, 20, 30 , 40, 50 ,60 , 70, 80 , 90, 95, 99]:
    split_feats = [_f for _f, _score, _ in correlation_scores if _score >= threshold]
    split_cat_feats = [_f for _f, _score, _ in correlation_scores if (_score >= threshold) & (_f in categorical_feats)]
    gain_feats = [_f for _f, _, _score in correlation_scores if _score >= threshold]
    gain_cat_feats = [_f for _f, _, _score in correlation_scores if (_score >= threshold) & (_f in categorical_feats)]
                                                                                             
    print('Results for threshold %3d' % threshold)
    split_results = score_feature_selection(df=data, train_features=split_feats, cat_feats=split_cat_feats, target=data['TARGET'])
    print('\t SPLIT : %.6f +/- %.6f' % (split_results[0], split_results[1]))
    gain_results = score_feature_selection(df=data, train_features=gain_feats, cat_feats=gain_cat_feats, target=data['TARGET'])
    print('\t GAIN  : %.6f +/- %.6f' % (gain_results[0], gain_results[1]))